In [2]:
data1 = open("D:/Workspace/PYTHON/NLP/Project2/Data/sentences_cleaned.txt", 'r', encoding='utf-8').read().split('\n')
data2 = open('D:/Workspace/PYTHON/NLP/Project2/Data/ExtraData/headlines.txt', 'r', encoding='utf-8').read().split('\n')

In [3]:
print('lengths:', len(data1), len(data2))
max_len1 = max(len(x) for x in data1)
max_len2 = max(len(x) for x in data2)
print('max lengths:', max_len1, max_len2)

lengths: 10271 111861
max lengths: 125 200


In [4]:
data= data1 # + data2
data = [line for line in data if line.strip()]  # Remove empty lines
print('Total lines:', len(data))
print("Max sentence length (chars):", max(len(line) for line in data))
print("Min sentence length (chars):", min(len(line) for line in data))
print("Max words in a line:", max(len(line.split()) for line in data))
print("Min words in a line:", min(len(line.split()) for line in data))




Total lines: 10270
Max sentence length (chars): 125
Min sentence length (chars): 8
Max words in a line: 25
Min words in a line: 3


In [5]:
chars = set(''.join(data))
chars.remove(' ') 
chars.add('_') 
chars = sorted(chars)
print('Total unique characters:', len(chars))
print(chars)

Total unique characters: 45
['_', '؟', 'ء', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'و', 'ي', 'ٹ', 'پ', 'چ', 'ڈ', 'ڑ', 'ژ', 'ک', 'گ', 'ں', 'ھ', 'ہ', 'ی', 'ے', 'ﷲ', 'ﷺ']


In [6]:
from collections import Counter

In [7]:
def build_word_freqs(lines):
    """Return Counter of words from list of lines."""
    freqs = Counter()
    for ln in lines:
        for w in ln.split():
            freqs[w ] += 1
    return freqs    

In [8]:
data_freq = build_word_freqs(data)
print('Total unique words:', len(data_freq))
data_freq.most_common()

Total unique words: 10350


[('ہے', 2920),
 ('کے', 2296),
 ('میں', 2282),
 ('کی', 2249),
 ('کا', 1744),
 ('سے', 1511),
 ('ہیں', 1231),
 ('کو', 1115),
 ('اور', 1022),
 ('اس', 963),
 ('نہیں', 870),
 ('یہ', 868),
 ('نے', 763),
 ('تو', 733),
 ('بھی', 722),
 ('پر', 710),
 ('ایک', 629),
 ('کیا', 628),
 ('ہو', 574),
 ('وہ', 560),
 ('کہ', 558),
 ('کر', 528),
 ('ہوں', 509),
 ('ان', 483),
 ('تھا', 400),
 ('ہی', 389),
 ('نہ', 368),
 ('پاکستان', 333),
 ('جو', 303),
 ('ہم', 263),
 ('کوئی', 262),
 ('لیے', 255),
 ('تھے', 249),
 ('گا', 247),
 ('کچھ', 242),
 ('اب', 238),
 ('اپ', 236),
 ('بہت', 225),
 ('گیا', 223),
 ('سب', 204),
 ('تھی', 199),
 ('بات', 199),
 ('اپنے', 194),
 ('کرتا', 187),
 ('کرنے', 182),
 ('رہے', 182),
 ('کسی', 181),
 ('رہا', 180),
 ('ہوتا', 180),
 ('ساتھ', 177),
 ('وقت', 171),
 ('اپنی', 169),
 ('جب', 167),
 ('کرنا', 166),
 ('اج', 165),
 ('لیکن', 163),
 ('پھر', 153),
 ('ہوا', 151),
 ('یا', 149),
 ('دیا', 148),
 ('گے', 148),
 ('ہر', 145),
 ('کام', 142),
 ('جائے', 138),
 ('سی', 136),
 ('ملک', 133),
 ('جا', 132),
 (

In [9]:
def get_stats(data): 
    """Compute frequency of symbol pairs in the vocabulary."""
    pairs = Counter()
    for word, freq in data.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += freq            
    return pairs

def merge_char_split_data(pair, char_split_data):
    """Merge the most frequent pair in the vocabulary."""
    a, b = pair
    new_symbol = a + b
    merged_char_split_data = {}
    for word, freq in char_split_data.items():
        new_word = word.replace(' '.join(pair), new_symbol)
        merged_char_split_data[new_word] = freq
    return merged_char_split_data

def train_bpe(vocab_size, char_split_data):
    """
    Train BPE with a stopping criterion based on vocabulary size.
    Uses '_' as space marker, no </w>.
    """
    vocab = []
    
    while len(vocab) < vocab_size:
        pairs = get_stats(char_split_data)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab.append(best)
        char_split_data = merge_char_split_data(best, char_split_data)
        print(f"New pair: {best}, Vocab size: {len(vocab)}")
    
    return vocab, char_split_data



In [10]:
char_split_data = {
    '_ ' + ' '.join(list(word)): freq 
    for word, freq in data_freq.items()
}

In [11]:

SPECIAL_TOKENS = ["<pad>","<sos>", "<eos>", "<mask>","<unk>"]  

In [12]:
print("Training BPE")
merges, corpus = train_bpe( 256- len(SPECIAL_TOKENS) - len(chars),char_split_data,)

Training BPE
New pair: ('_', 'ک'), Vocab size: 1
New pair: ('_', 'ا'), Vocab size: 2
New pair: ('_', 'ہ'), Vocab size: 3
New pair: ('_', 'م'), Vocab size: 4
New pair: ('ی', 'ں'), Vocab size: 5
New pair: ('_', 'ب'), Vocab size: 6
New pair: ('_', 'س'), Vocab size: 7
New pair: ('_', 'ت'), Vocab size: 8
New pair: ('_', 'پ'), Vocab size: 9
New pair: ('_', 'ن'), Vocab size: 10
New pair: ('_ک', 'ی'), Vocab size: 11
New pair: ('_ہ', 'ے'), Vocab size: 12
New pair: ('ا', 'ن'), Vocab size: 13
New pair: ('_', 'ج'), Vocab size: 14
New pair: ('_', 'د'), Vocab size: 15
New pair: ('ا', 'ر'), Vocab size: 16
New pair: ('و', 'ر'), Vocab size: 17
New pair: ('_ک', 'ے'), Vocab size: 18
New pair: ('_م', 'یں'), Vocab size: 19
New pair: ('_ہ', 'و'), Vocab size: 20
New pair: ('ی', 'ا'), Vocab size: 21
New pair: ('_ک', 'ا'), Vocab size: 22
New pair: ('_', 'و'), Vocab size: 23
New pair: ('_', 'ر'), Vocab size: 24
New pair: ('_', 'ل'), Vocab size: 25
New pair: ('_', 'گ'), Vocab size: 26
New pair: ('_ک', 'ر'), Voca

In [13]:
merges= [a+b for a,b in merges]
print("Subord numbers:", len(merges))
vocab = list(chars) + merges + SPECIAL_TOKENS
print("Vocabulary size:", len(vocab))

Subord numbers: 206
Vocabulary size: 256


In [14]:
def token2id(tokens):
    token2id = {}
    next_id = 0
    # Assign first 4 indices to special tokens
    for tok in SPECIAL_TOKENS:
        token2id[tok] = next_id
        next_id += 1
    # Assign remaining tokens (excluding special tokens)
    for tok in sorted(tokens):
        if tok in token2id:
            continue
        token2id[tok] = next_id
        next_id += 1
    return token2id

In [15]:
vocab=token2id(vocab)
print("Token to ID mapping:", vocab)

Token to ID mapping: {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<mask>': 3, '<unk>': 4, '_': 5, '_ا': 6, '_اج': 7, '_اس': 8, '_اع': 9, '_اور': 10, '_اپ': 11, '_اپنی': 12, '_اپنے': 13, '_ایس': 14, '_ایک': 15, '_ب': 16, '_بات': 17, '_بڑ': 18, '_بھ': 19, '_بھی': 20, '_بہ': 21, '_بہت': 22, '_ت': 23, '_تع': 24, '_تو': 25, '_تک': 26, '_تھ': 27, '_تھا': 28, '_تھے': 29, '_ج': 30, '_جا': 31, '_جب': 32, '_جس': 33, '_جو': 34, '_ح': 35, '_حک': 36, '_خ': 37, '_خو': 38, '_د': 39, '_دوس': 40, '_دیا': 41, '_دیکھ': 42, '_ذ': 43, '_ر': 44, '_رو': 45, '_رکھ': 46, '_رہ': 47, '_رہا': 48, '_رہے': 49, '_ز': 50, '_س': 51, '_ساتھ': 52, '_سب': 53, '_سو': 54, '_سک': 55, '_سے': 56, '_ش': 57, '_شک': 58, '_ص': 59, '_ض': 60, '_ط': 61, '_ع': 62, '_عم': 63, '_غ': 64, '_ف': 65, '_فی': 66, '_ق': 67, '_ل': 68, '_لو': 69, '_لوگ': 70, '_لگ': 71, '_لیکن': 72, '_لیے': 73, '_م': 74, '_مجھ': 75, '_مح': 76, '_مس': 77, '_مش': 78, '_مع': 79, '_مق': 80, '_مل': 81, '_ملک': 82, '_من': 83, '_مو': 84, '_میں': 85, '_ن': 86, '_نظ': 87, '_نہ'

In [16]:
import json

with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab, f, ensure_ascii=False, indent=2)
